## Produce daily Origin-Destination trip matrix for NYC resident cbgs
## Include time away from home buckets, and various device counts

##### Output is master excel table for origin NYC boroughs by day and destinations, time away from home, and device behavior

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# read nyc origin cbgs
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')

In [6]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

## RUNNING FOR 1 MONTH ONLY, Y-o-Y COMPARISON

In [7]:
# set date variables, in this case, a test month for 2 different years
month = "08"
years = ["2019","2020"]
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
d1 = 0
d2 = 31

# If running all days and months - replace range and adjust for loop to run through lists below:
#monthList =["01","02","03","04","05","06","07","08"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31,31] 

In [8]:
## Iterate and create pivot for home county to destination county (in region + outside)
frames = [] 
for y in years:
    for i in range(d1,d2):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')
            
            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #separate home trips from other trips for later aggregation
            df['is_home'] = df.apply(lambda x: x['orig_cbg']==x['destination_cbg'],axis=1)
            #id destinations by county fips
            df['dest_stco_all'] = df['destination_cbg'].str[:5]
            #id 31cr counties and all others outside
            df['dest_stco_reg'] = df['dest_stco_all'] 
            df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR' 
            df['dest_sub'] = df['dest_stco_all'].map(sub).fillna('O31CR')

            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','dest_cbg_count','dest_stco_reg','dest_sub','is_home']]
            frames.append(dff) 

2019-08-01
2019-08-02
2019-08-03
2019-08-04
2019-08-05
2019-08-06
2019-08-07
2019-08-08
2019-08-09
2019-08-10
2019-08-11
2019-08-12
2019-08-13
2019-08-14
2019-08-15
2019-08-16
2019-08-17
2019-08-18
2019-08-19
2019-08-20
2019-08-21
2019-08-22
2019-08-23
2019-08-24
2019-08-25
2019-08-26
2019-08-27
2019-08-28
2019-08-29
2019-08-30
2019-08-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-09
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-16
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-30
2020-08-31


In [9]:
df_dest = pd.concat(frames)
df_dest = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d','orig_stco'],columns=['dest_stco_reg','is_home'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [10]:
df_dest.head(10)

dest_cbg_count                                            \
dest_stco_reg                 09001 09005 09009 34003 34013 34017 34019 34021   
is_home                       False False False False False False False False   
date_y-m-d orig_stco                                                            
2019-08-01 36005                366    16    98   642   316   414     6    35   
           36047                201    42   205   474   786   946    38    94   
           36061                304    55   142   888   678   861    10   110   
           36081                312    21   111   819   498   671    12    61   
           36085                 33     2    19   245   354   574    13    73   
2019-08-02 36005                408    23   143   636   344   454    13    55   
           36047                238    61   239   529   772  1030    58   132   
           36061                491   169   236   973   677   876    21   116   
           36081                424    21   157   771   475   672    28   108   
           36085                 61     3    21   234   398   549    53    55   

                                                                            \
dest_stco_reg        34023 34025 34027 34029 34031 34035 34037 34039 34041   
is_home              False False False False False False False False False   
date_y-m-d orig_stco                                                         
2019-08-01 36005       185    54    64   128   192    20    11   136     9   
           36047       479   289   142   231   154    78    29   349    23   
           36061       227   223   121   118   112    68    20   236     8   
           36081       272   118    65   113   118    60    44   253    22   
           36085      1052   815    96   703    45   104    49   465    14   
2019-08-02 36005       183    82    67   150   200    29    23   160    18   
           36047       540   528   179   310   147    90    55   379    37   
           36061       244   517   132   214   167    86    61   240    17   
           36081       359   176   101   156   131    84    48   232    27   
           36085      1233   971    96   864    78    93    93   492    16   

                                                                               \
dest_stco_reg         36005        36027   36047          36059  36061          
is_home               False   True False   False    True  False  False   True   
date_y-m-d orig_stco                                                            
2019-08-01 36005      63305  61072   134    4381       0    536  24059      0   
           36047       3487      0    98  124530  102584   2367  32617      0   
           36061       6070      0   108    5203       0    794  75608  55565   
           36081       3391      0   105   16223       0  14270  31557      0   
           36085        337      0     4    6630       0    242   5578      0   
2019-08-02 36005      64922  60229   117    4432       0    617  23893      0   
           36047       3453      0   169  127938  100935   2572  31039      0   
           36061       6589      0   198    5552       0   1034  73490  53954   
           36081       3517      0   117   16716       0  14609  30358      0   
           36085        433      0    21    6707       0    232   5281      0   

                                                                            \
dest_stco_reg        36071 36079   36081          36085        36087 36103   
is_home              False False   False    True  False   True False False   
date_y-m-d orig_stco                                                         
2019-08-01 36005       162    49    4123       0    150      0   235   288   
           36047       152    26   13602       0   1655      0   162  1077   
           36061       151    39    4977       0    301      0   242  1216   
           36081       182    51  113676  104785    376      0   155  2601   
           36085        37     4  

In [11]:
#Run an iteration to unpack the bucketed time away from home

In [12]:
frames = [] 
for y in years:
    for i in range(d1,d2):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json bucketed time away from home
            df = sgpy.unpack_json_and_merge(df, json_column='bucketed_away_from_home_time', key_col_name='away_from_home_time', value_col_name='away_count')
            
            ##Make new columns
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','away_from_home_time','away_count']]
            frames.append(dff) 

2019-08-01
2019-08-02
2019-08-03
2019-08-04
2019-08-05
2019-08-06
2019-08-07
2019-08-08
2019-08-09
2019-08-10
2019-08-11
2019-08-12
2019-08-13
2019-08-14
2019-08-15
2019-08-16
2019-08-17
2019-08-18
2019-08-19
2019-08-20
2019-08-21
2019-08-22
2019-08-23
2019-08-24
2019-08-25
2019-08-26
2019-08-27
2019-08-28
2019-08-29
2019-08-30
2019-08-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-09
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-16
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-30
2020-08-31


In [13]:
df_away = pd.concat(frames)
df_away = pd.pivot_table(df_away,values=['away_count'],index=['date_y-m-d','orig_stco'],columns=['away_from_home_time'],aggfunc=np.sum,fill_value=0,margins=True)

In [14]:
df_away.head(10)

away_count                                            \
away_from_home_time   1081-1200 1201-1320 121-180 1321-1440 181-240 21-45   
date_y-m-d orig_stco                                                        
2019-08-01 36005            816       642    3764       629    3296  4012   
           36047           1512      1098    6572      1054    5845  6725   
           36061           1168       781    3770       621    3359  3872   
           36081           1363      1070    6341      1043    5426  6159   
           36085            497       365    2011       341    1850  1662   
2019-08-02 36005            939       616    4065       617    3374  4173   
           36047           1798      1220    6939      1134    5864  7124   
           36061           1374       946    4159       646    3458  4068   
           36081           1586      1136    6725      1078    5825  6219   
           36085            533       415    2139       359    2012  1789   

                                                                            \
away_from_home_time  241-300 301-360 361-420 421-480 46-60 481-540 541-600   
date_y-m-d orig_stco                                                         
2019-08-01 36005        2647    2478    2417    2616  1796    2509    1999   
           36047        4775    4449    4566    4831  3106    4834    3793   
           36061        2639    2359    2511    2737  1699    2732    2200   
           36081        4534    4465    4434    4889  2733    5338    4332   
           36085        1565    1541    1351    1597   761    1582    1171   
2019-08-02 36005        2891    2535    2344    2501  1795    2415    1925   
           36047        4820    4580    4594    4739  3257    4537    3697   
           36061        2805    2570    2500    2611  1880    2526    2049   
           36081        4721    4418    4449    4836  2825    5013    4157   
           36085        1672    1463    1387    1434   783    1454    1126   

                                                                             \
away_from_home_time  601-660 61-120 661-720 721-840 841-960 961-1080    <20   
date_y-m-d orig_stco                                                          
2019-08-01 36005        1394   5189     955    1256     882      781  33411   
           36047        2701   8643    1994    2299    1694     1457  50295   
           36061        1660   4987    1280    1663    1206     1181  29930   
           36081        2961   8429    2108    2242    1637     1355  47242   
           36085         771   2547     576     630     442      443  11596   
2019-08-02 36005        1567   5239    1192    1451     926      879  32655   
           36047        2821   9192    2089    2480    1831     1617  50240   
           36061        1656   5174    1268    1765    1351     1439  29384   
           36081        3015   8635    2219    2617    1681     1468  46286   
           36085         796   2656     610     721     562      567  11360   

                              
away_from_home_time      All  
date_y-m-d orig_stco          
2019-08-01 36005       73489  
           36047      122243  
           36061       72355  
           36081      118101  
           36085       33299  
2019-08-02 36005       74099  
           36047      124573  
           36061       73629  
           36081      118909  
           36085       33838

In [15]:
# Run another iteration for just counts of devices
#'device_count','completely_home_device_count','part_time_work_behavior_devices','full_time_work_behavior_devices',

In [16]:
frames = [] 
for y in years:
    for i in range(d1,d2):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")
            
            ##Make new columns
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','device_count','completely_home_device_count','part_time_work_behavior_devices','full_time_work_behavior_devices']]
            frames.append(dff)

2019-08-01
2019-08-02
2019-08-03
2019-08-04
2019-08-05
2019-08-06
2019-08-07
2019-08-08
2019-08-09
2019-08-10
2019-08-11
2019-08-12
2019-08-13
2019-08-14
2019-08-15
2019-08-16
2019-08-17
2019-08-18
2019-08-19
2019-08-20
2019-08-21
2019-08-22
2019-08-23
2019-08-24
2019-08-25
2019-08-26
2019-08-27
2019-08-28
2019-08-29
2019-08-30
2019-08-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-09
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-16
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-30
2020-08-31


In [17]:
df_device = pd.concat(frames)
df_device = df_device.groupby(['date_y-m-d','orig_stco']).agg({'device_count':np.sum,'completely_home_device_count':np.sum,\
                                                    'part_time_work_behavior_devices':np.sum,\
                                                    'full_time_work_behavior_devices':np.sum})

In [18]:
#Run a concat of all three tables to make master table with origin/day index

In [19]:
df_master = pd.concat([df_device,df_dest,df_away],axis=1)
df_master = df_master.reset_index()

In [20]:
df_master.to_excel(f'output/dara/{month}-_bigtable.xlsx')


In [21]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')

Run time - 14542.98825097084 seconds
